# Machine Learning (Classificação, Regressão, Clustering e Sistema de Recomendação)

### Descriçao

---
1. O objetivo deste notebook é aprender as 4 principais tarefas de ML (Classificação, Regressão, Agrupagamentos, Sistema de Recomendação)
2. Vamos construir um conjunto de dados baseado no universo do Harry Potter
3. O problema consiste em prever, baseado no histórico escolar do aluno, se ele está apto a entrar na casa que ele se candidatou.
---

# Instalação dos pacotes

In [1]:
!pip install pandas numpy scikit-learn Faker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.3 MB/s eta 0:00:00


# Importando as Bibliotecas

In [2]:
# Bibliotecas para manipulação e geração de dados
import pandas as pd  # Manipulação de dados tabulares (DataFrames)
import numpy as np  # Operações com arrays e matrizes numéricas
import random  # Geração de números aleatórios
from faker import Faker  # Geração de dados fictícios (ex: nomes, endereços)

# Bibliotecas do Scikit-Learn para Machine Learning
from sklearn.model_selection import train_test_split  # Divisão dos dados em conjuntos de treino e teste
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder  # Pré-processamento para padronização e codificação de variáveis
from sklearn.compose import ColumnTransformer  # Aplicação de transformações diferentes para colunas específicas

# Métricas para avaliação de modelos
from sklearn.metrics import (
  mean_absolute_error,  # Erro absoluto médio
  mean_squared_error,  # Erro quadrático médio
  r2_score,  # Coeficiente de determinação (R²)
  accuracy_score,  # Acurácia (para classificadores)
  silhouette_score  # Pontuação para avaliação de agrupamentos (clustering)
)

# Modelos de machine learning
from sklearn.ensemble import RandomForestClassifier  # Classificador baseado em floresta de árvores de decisão
from sklearn.linear_model import LogisticRegression  # Modelo de regressão logística
from sklearn.cluster import KMeans  # Algoritmo de agrupamento (clustering)
from sklearn.neighbors import NearestNeighbors  # Algoritmo usado para sistemas de recomendação (vizinhos mais próximos)

# Utilitários
import joblib  # Salvamento e carregamento de modelos treinados
import os  # Manipulação de diretórios e arquivos no sistema operacional

# Construindo o Dataset Faker



In [3]:
def generate(num):  # Recebe um valor numérico que define a quantidade de linhas (alunos) a serem geradas
  fake = Faker()  # Instancia um objeto Faker para gerar dados fictícios
  data = []  # Lista onde serão armazenados os dados gerados

  for i in range(num):  # Loop para gerar os dados de cada aluno
    aluno = fake.name()  # Gera um nome fictício
    idade = random.randint(11, 39)  # Gera uma idade aleatória entre 11 e 39 anos
    casas = random.choice(['Grifinória', 'Sonserina', 'Corvinal', 'Lufa-lufa'])  # Escolhe uma casa aleatoriamente
    ano_de_ingresso = random.randint(2024 - idade, 2024)  # Define o ano de ingresso com base na idade

    nota_1 = round(random.uniform(0, 100), 2)  # Gera uma nota aleatória entre 0 e 100 (ensino fundamental)
    # Se nota_1 for menor que 50, nota_2 (ensino médio) será 0. Caso contrário, gera uma nova nota aleatória
    nota_2 = 0 if nota_1 < 50 else round(random.uniform(0, 100), 2)
    # Se nota_2 for 0, a nota final será igual à nota_1; caso contrário, é a média entre as duas
    nota_final = nota_1 if nota_2 == 0 else round((nota_1 + nota_2) / 2, 2)
    # Define a classificação final com base na nota final
    label_alvo = 'reprovado' if nota_final < 50 else 'aprovado'

    # Cria o dicionário com os dados do aluno
    row = {
      'aluno': aluno,
      'idade': idade,
      'casas': casas,
      'ano_de_ingresso': ano_de_ingresso,
      'ensino_fundamental': nota_1,
      'ensino_medio': nota_2,
      'nota_final': nota_final,
      'label_alvo': label_alvo
    }

    data.append(row)  # Adiciona o dicionário à lista

  df = pd.DataFrame(data)  # Converte a lista de dicionários em um DataFrame
  return df  # Retorna o DataFrame gerado

# Geração de um DataFrame com 1000 alunos
df = generate(1000)
df  # Exibe o DataFrame

,aluno,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,label_alvo
0,Katelyn Baker,27,Lufa-lufa,2021,80.56,0.67,40.62,reprovado
1,Brandy Allen,38,Sonserina,2006,48.66,0.00,48.66,reprovado
2,Dr. Ronald Saunders,30,Lufa-lufa,1995,24.94,0.00,24.94,reprovado
3,Anna Mitchell,31,Grifinória,2019,39.11,0.00,39.11,reprovado
4,Anthony Carter,19,Lufa-lufa,2013,79.04,82.89,80.97,aprovado
...,...,...,...,...,...,...,...,...
995,Wyatt Guzman,18,Grifinória,2010,91.60,96.94,94.27,aprovado
996,William Neal,17,Lufa-lufa,2013,98.54,84.50,91.52,aprovado
997,Rebecca Alexander,16,Sonserina,2015,88.91,92.90,90.91,aprovado
998,Heather Delacruz,26,Lufa-lufa,2011,51.06,39.67,45.37,reprovado


# Classificação

#### **Objetivo**
1. Na classificação, nosso objetivo é prever ou descrever uma variavel categorica (string)
2. Vamos usar a coluna catégorica [label_alvo] para nosso estudo.

In [4]:
# Cria uma cópia do DataFrame original removendo a coluna 'aluno',
# já que nomes não são relevantes para análise e podem ser considerados dados sensíveis
df_new = df.drop(['aluno'], axis=1)

df_new  # Exibe o novo DataFrame sem a coluna 'aluno'

,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,label_alvo
0,27,Lufa-lufa,2021,80.56,0.67,40.62,reprovado
1,38,Sonserina,2006,48.66,0.00,48.66,reprovado
2,30,Lufa-lufa,1995,24.94,0.00,24.94,reprovado
3,31,Grifinória,2019,39.11,0.00,39.11,reprovado
4,19,Lufa-lufa,2013,79.04,82.89,80.97,aprovado
...,...,...,...,...,...,...,...
995,18,Grifinória,2010,91.60,96.94,94.27,aprovado
996,17,Lufa-lufa,2013,98.54,84.50,91.52,aprovado
997,16,Sonserina,2015,88.91,92.90,90.91,aprovado
998,26,Lufa-lufa,2011,51.06,39.67,45.37,reprovado


In [5]:
# Separação das variáveis independentes (features) e da variável dependente (rótulo/target)

X = df_new.iloc[:, :-1]  # Seleciona todas as colunas, exceto a última ('label_alvo'), como features
y = df_new['label_alvo']  # Define a coluna 'label_alvo' como variável alvo (target)

In [6]:
# Codificação de variáveis categóricas para formato numérico

label_encoder = LabelEncoder()  # Instancia o codificador de rótulos

# Codifica a coluna 'casas' (variável categórica) para valores numéricos
X['casas'] = label_encoder.fit_transform(X['casas'])

# Codifica os valores da variável alvo 'label_alvo' (aprovado/reprovado) em números (0 e 1)
y = label_encoder.fit_transform(y)

In [7]:
X.head()  # Exibe as primeiras linhas do DataFrame X, que agora contém variáveis numéricas

,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final
0,27,2,2021,80.56,0.67,40.62
1,38,3,2006,48.66,0.00,48.66
2,30,2,1995,24.94,0.00,24.94
3,31,1,2019,39.11,0.00,39.11
4,19,2,2013,79.04,82.89,80.97


In [8]:
# As categorias transformadas em números podem induzir o modelo ao erro,
# pois ele pode interpretar valores maiores como superiores — o que não é verdade para variáveis categóricas nominais.
# Para evitar isso, usamos codificação one-hot.

# --- Método 1: Utilizando get_dummies do pandas ---
# Converte a coluna 'casas' em variáveis dummies (colunas binárias) e força o tipo int para consistência
f = pd.get_dummies(X, columns=['casas']).astype(int)


# --- Método 2: Utilizando OneHotEncoder do Scikit-learn ---
# Usamos ColumnTransformer para aplicar o OneHotEncoder apenas à coluna 'casas' (índice 1),
# mantendo as demais colunas inalteradas (remainder='passthrough')
ct = ColumnTransformer(
  [("Casas", OneHotEncoder(), [1])],  # Nome, método e índice da coluna a codificar
  remainder='passthrough'  # Mantém as outras colunas sem alterações
)

# Aplica a transformação
o = ct.fit_transform(X)

In [9]:
# Exibe as primeiras 5 linhas do DataFrame 'f' (versão de X com codificação one-hot via get_dummies)
f.head()

,idade,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,casas_0,casas_1,casas_2,casas_3
0,27,2021,80,0,40,0,0,1,0
1,38,2006,48,0,48,0,0,0,1
2,30,1995,24,0,24,0,0,1,0
3,31,2019,39,0,39,0,1,0,0
4,19,2013,79,82,80,0,0,1,0


In [10]:
# Divisão dos dados em conjuntos de treino e teste

X_train, X_test, y_train, y_test = train_test_split(
  o,              # Features codificadas com OneHotEncoder
  y,              # Labels codificados (0 = reprovado, 1 = aprovado)
  test_size=0.2,  # 20% dos dados serão usados para teste
  random_state=42 # Semente para garantir reprodutibilidade dos resultados
)

In [11]:
# Inicialização e treinamento do modelo de classificação usando Random Forest

forest_classifier = RandomForestClassifier(
  n_estimators=20,  # Número de árvores na floresta (quanto maior, mais robusto, mas mais custoso)
  random_state=42   # Semente para reprodutibilidade
)

# Treinamento do modelo com os dados de treino
forest_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=20, random_state=42)

In [12]:
# Exibe o tamanho (quantidade de amostras) dos conjuntos de treino e teste

print(f"O tamanho do treino: {len(X_train)}")  # Quantidade de amostras usadas para treinar o modelo
print(f"O tamanho do teste: {len(X_test)}")    # Quantidade de amostras usadas para testar o modelo

O tamanho do treino: 800
O tamanho do teste: 200


In [13]:
from sklearn.metrics import (
  confusion_matrix,
  classification_report,
  roc_auc_score
)

# Previsão com os dados de teste
y_pred = forest_classifier.predict(X_test)

# Acurácia: proporção de acertos
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy:.2f}')

# Matriz de confusão: mostra os acertos e erros de classificação
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

# Relatório de classificação: mostra precisão, recall, f1-score e suporte
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# ROC AUC Score: área sob a curva ROC (apenas para classificação binária)
# Verifica se o problema é binário antes de calcular
if len(np.unique(y_test)) == 2:
  y_prob = forest_classifier.predict_proba(X_test)[:, 1]  # Probabilidades da classe positiva
  auc = roc_auc_score(y_test, y_prob)
  print(f"ROC AUC Score: {auc:.2f}")

Acurácia do modelo: 1.00

Matriz de Confusão:
[[ 79   0]
 [  0 121]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        79
           1       1.00      1.00      1.00       121

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

ROC AUC Score: 1.00


#### Salvando o modelo

In [15]:
# Criação de uma pasta para armazenar o modelo treinado
folder = 'classificacao'

os.mkdir(folder)  # Cria a pasta 'classificacao' (certifique-se de que ela ainda não exista)

# Salva o modelo Random Forest treinado em um arquivo .pkl usando joblib
with open(f'{folder}/aprovacao_random_forest.pkl', 'wb') as f:
    joblib.dump(forest_classifier, f)

### Reutilizando o modelo

In [16]:
# Carrega o modelo Random Forest salvo anteriormente
rf_classifier = joblib.load(f'{folder}/aprovacao_random_forest.pkl')

In [17]:
# Testando o modelo com um novo aluno

nome_candidado = 'Daniel Ferreira'  # Nome fictício do novo aluno
idade = 35
casa_pretendida = 'Lufa-lufa'
ano_de_ingresso = 2024
ensino_fundamental = 58.9
ensino_medio = 10.0
casas_0 = 1
casas_1 = 0
casas_2 = 0
casas_3 = 0

# Calcula a média das notas
nota_final = (ensino_fundamental + ensino_medio) / 2

# Cria o vetor com os dados numéricos (sem incluir 'casas' ainda)
X_novo = [[idade, ano_de_ingresso, ensino_fundamental, ensino_medio, nota_final, casas_0, casas_1, casas_2, casas_3]]

# Previsão com o modelo carregado
y_pred = rf_classifier.predict(X_novo)

# Decodifica o resultado numérico de volta para "aprovado" ou "reprovado"
resultado = label_encoder.inverse_transform(y_pred)[0]

# Gera a resposta final formatada
resposta = f"O aluno {nome_candidado} pretendido a casa {casa_pretendida} foi {resultado.upper()}"

In [18]:
resposta

'O aluno Daniel Ferreira pretendido a casa Lufa-lufa foi REPROVADO'

# Regressão

#### **Objetivo**
1. Em regressão eu tento prever ou descrever uma categoria númerica (int)
2. Neste caso iremos criar uma regressão para podermos prever, com base nas notas final quais sao as chances do aluno de entrar na casa pretendida.




In [19]:
# Gerando um novo conjunto de dados fictício com 1000 alunos
# A função generate() retorna um DataFrame com dados simulados para:
# - Nome do aluno (falso)
# - Idade
# - Casa (Grifinória, Sonserina, Corvinal, Lufa-lufa)
# - Ano de ingresso
# - Nota do ensino fundamental
# - Nota do ensino médio (ou 0 se reprovado na etapa anterior)
# - Nota final (média entre as duas)
# - Label de aprovação ou reprovação

df = generate(1000)  # Gera 1000 registros
df  # Exibe o DataFrame gerado

,aluno,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,label_alvo
0,Tammy Bruce,30,Sonserina,2010,72.85,6.82,39.83,reprovado
1,Michael Jacobs,14,Corvinal,2019,35.71,0.00,35.71,reprovado
2,Barbara Chandler,38,Lufa-lufa,2016,70.02,83.60,76.81,aprovado
3,Lisa Anthony,35,Lufa-lufa,1994,6.72,0.00,6.72,reprovado
4,Danielle Allen,27,Corvinal,2011,98.12,58.88,78.50,aprovado
...,...,...,...,...,...,...,...,...
995,Savannah Reyes,28,Sonserina,2014,90.36,37.98,64.17,aprovado
996,Alicia Ward,26,Grifinória,2014,50.78,28.45,39.62,reprovado
997,John Nguyen MD,25,Sonserina,2011,57.65,20.19,38.92,reprovado
998,Dana Wheeler,14,Sonserina,2015,13.36,0.00,13.36,reprovado


In [20]:
# Transformando variáveis categóricas ('label_alvo' e 'casas') em valores numéricos
# Isso é necessário para que algoritmos de machine learning consigam processar os dados

label_encoder = LabelEncoder()  # Instancia o codificador de rótulos

# Converte os valores 'aprovado'/'reprovado' da coluna 'label_alvo' em 0 e 1
df['label_alvo'] = label_encoder.fit_transform(df['label_alvo'])

# Converte os nomes das casas (ex: 'Grifinória', 'Sonserina') em valores numéricos
df['casas'] = label_encoder.fit_transform(df['casas'])

In [21]:
# Separando os dados para um modelo supervisionado (classificação)
# X contém as features (variáveis independentes) que serão usadas para prever a aprovação
# y contém o rótulo (variável dependente), que indica se o aluno foi aprovado ou reprovado

X = df[['nota_final', 'casas']]  # Seleciona apenas as colunas relevantes como entrada do modelo
y = df['label_alvo']             # Coluna alvo: 0 (reprovado) ou 1 (aprovado)

In [22]:
# Dividindo o conjunto de dados em treino e teste
# - 80% dos dados serão usados para treinar o modelo (X_train, y_train)
# - 20% dos dados serão usados para testar o modelo (X_test, y_test)
# - random_state garante que a divisão seja sempre a mesma ao rodar o código novamente (reprodutibilidade)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Criando e treinando um modelo de Regressão Logística
# Esse modelo será usado para prever se o aluno será aprovado ou reprovado
# com base nas variáveis: nota_final e casa (codificada numericamente)

regression_model = LogisticRegression()         # Instancia o modelo de regressão logística
regression_model.fit(X_train, y_train)          # Treina o modelo com os dados de treino

LogisticRegression()

In [24]:
from sklearn.metrics import classification_report

# Fazendo previsões com os dados de teste
y_pred = regression_model.predict(X_test)

# Gerando o relatório com precisão, recall e f1-score
relatorio = classification_report(y_test, y_pred, target_names=['Reprovado', 'Aprovado'])

print("Relatório de desempenho do modelo:\n")
print(relatorio)

Relatório de desempenho do modelo:

              precision    recall  f1-score   support

   Reprovado       1.00      0.97      0.99        70
    Aprovado       0.98      1.00      0.99       130

    accuracy                           0.99       200
   macro avg       0.99      0.99      0.99       200
weighted avg       0.99      0.99      0.99       200



#### Salvando o modelo

In [25]:
# Salvando o modelo de regressão logística treinado
# Isso permite reutilizar o modelo posteriormente sem precisar reentreinar

folder = 'regressao'  # Nome da pasta onde o modelo será salvo

os.mkdir(folder)  # Cria a pasta (se ainda não existir)

# Abre o arquivo em modo escrita binária e salva o modelo usando joblib
with open(f'{folder}/regression_logistic.pkl', 'wb') as f:
    joblib.dump(regression_model, f)

### Reutilizando o modelo

In [26]:
# Carregando o modelo de regressão logística salvo anteriormente
# Isso evita a necessidade de treinar o modelo novamente

lg = joblib.load(f'{folder}/regression_logistic.pkl')  # 'lg' é a variável que armazenará o modelo carregado

In [27]:
# Simulação de previsão com o modelo de regressão logística

aluno = 'Daniel Ferreira'
casa_pretendida = 'Grifinória'
nota_final = 50

# Transformando a casa (categórica) para valor numérico com o mesmo LabelEncoder usado no treino
transfor_casa = label_encoder.transform([casa_pretendida])[0]

# Preparando os dados de entrada (deve seguir o mesmo formato usado no treino)
entrada = [[nota_final, transfor_casa]]

# Predict_proba retorna a probabilidade de cada classe (Reprovado, Aprovado)
# Ex: [0.3, 0.7] → 30% de chance de reprovar, 70% de chance de aprovar
resultado = lg.predict_proba(entrada)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [28]:
# Exibindo o resultado da previsão de forma legível

print(f"Resultado final: {resultado}")  # Mostra as probabilidades de [reprovado, aprovado]
print(f"O aluno {aluno} tem {round(resultado[0][0], 2) * 100}% de chance de entrar na {casa_pretendida}")

Resultado final: [[0.33316616 0.66683384]]
O aluno Daniel Ferreira tem 33.0% de chance de entrar na Grifinória


# Agrupamentos (clustering)

#### **Objetivo**
1. Em agrupamentos eu não tento prever uma categoria especifica, tento agrupar os dados de acordo com caracteristicas em comum.
2. Neste nosso caso vamos criar um cluster para entender como os alunos realmente estão dividos entre si, pelas notas de ensino, pelo ano de ingresso, ou pela casa que se candidataram.

In [29]:
# Gera um DataFrame com 100.000 registros utilizando a função generate
df = generate(100000)

# Exibe o DataFrame gerado
df

,aluno,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,label_alvo
0,Valerie Sanders,19,Grifinória,2023,89.84,43.64,66.74,aprovado
1,Alexandria Wall,15,Sonserina,2024,7.27,0.00,7.27,reprovado
2,Daniel Shepard,22,Lufa-lufa,2011,67.89,25.81,46.85,reprovado
3,Patrick Garcia,36,Lufa-lufa,2006,88.70,48.85,68.78,aprovado
4,Sarah Vega,31,Lufa-lufa,1999,39.60,0.00,39.60,reprovado
...,...,...,...,...,...,...,...,...
99995,Alexander White,14,Sonserina,2023,10.74,0.00,10.74,reprovado
99996,Patricia Kent,28,Corvinal,1996,38.52,0.00,38.52,reprovado
99997,David Turner,24,Sonserina,2023,50.01,4.62,27.31,reprovado
99998,Mercedes Combs,28,Corvinal,2006,51.34,37.56,44.45,reprovado


In [30]:
# Remove a coluna 'aluno' do DataFrame
X = df.drop(['aluno'], axis=1)

# Filtra apenas as linhas onde a coluna 'label_alvo' é igual a 'aprovado'
X = X[X['label_alvo'] == 'aprovado']

# Remove a coluna 'label_alvo', pois agora todos os registros são apenas de alunos aprovados
X = X.drop(['label_alvo'], axis=1)

# Exibe o número de linhas e colunas do novo DataFrame
X.shape

(37482, 6)

In [31]:
# Cria um objeto LabelEncoder para converter categorias em números
le = LabelEncoder()

# Aplica o LabelEncoder na coluna 'casas', transformando categorias em valores numéricos
X['casas'] = le.fit_transform(X['casas'])

In [32]:
# Cria um objeto StandardScaler para normalizar os dados
scaler = StandardScaler()

# Ajusta o scaler aos dados e transforma (normaliza) o DataFrame X
X_scaled = scaler.fit_transform(X)

In [33]:
# Cria um modelo KMeans para agrupamento, definindo 4 clusters
km = KMeans(n_clusters=4, random_state=42)

# Ajusta o modelo aos dados normalizados e atribui um cluster para cada amostra
clusters = km.fit_predict(X_scaled)

In [35]:
# Adiciona a informação dos clusters como uma nova coluna chamada 'grupo' no DataFrame original
X['grupo'] = clusters

In [36]:
X

,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,grupo
0,19,1,2023,89.84,43.64,66.74,1
3,36,2,2006,88.70,48.85,68.78,3
8,21,2,2016,50.59,62.73,56.66,2
12,22,0,2002,58.38,62.81,60.59,2
14,16,1,2020,91.54,62.29,76.92,0
...,...,...,...,...,...,...,...
99983,19,1,2013,70.54,75.36,72.95,2
99987,28,2,2021,68.58,45.72,57.15,2
99988,37,3,1992,63.06,75.00,69.03,3
99989,35,1,2023,97.91,90.00,93.95,0


In [37]:
X['grupo'].unique()

array([1, 3, 2, 0], dtype=int32)

In [39]:
# Calcula a inércia (soma das distâncias quadradas dos pontos aos seus centróides)
inertia = km.inertia_

# Calcula o coeficiente de silhueta, que mede a qualidade dos clusters formados
silhouette = silhouette_score(X_scaled, clusters)

In [40]:
# Imprime o valor da inércia formatado com 2 casas decimais
print(f'Inércia: {inertia:.2f}')

# Imprime o valor do coeficiente de silhueta formatado com 4 casas decimais
print(f'Coeficiente de Silhueta: {silhouette:.4f}')

Inércia: 130147.19
Coeficiente de Silhueta: 0.1938


### Salvando o modelo

In [41]:
# Define o nome da pasta onde o modelo será salvo
folder = 'clustering'

# Cria a pasta (diretório) chamada 'clustering'
os.mkdir(folder)

# Abre (ou cria) um arquivo 'clustering.pkl' dentro da pasta para escrita binária
with open(f'{folder}/clustering.pkl', 'wb') as f:
    # Salva (serializa) o modelo KMeans treinado usando joblib
    joblib.dump(km, f)

### Testando com novos dados

In [42]:
#importando o modelo
km = joblib.load(f'{folder}/clustering.pkl')

In [43]:
# Gera uma nova base de dados maior com 10.000 registros
df_new = generate(10000)

# Remove as colunas 'aluno' e 'label_alvo' do DataFrame
X = df_new.drop(['aluno'], axis=1)

# Pré-processamento:

# Converte a coluna 'casas' em valores numéricos usando LabelEncoder
le = LabelEncoder()
X['casas'] = le.fit_transform(X['casas'])

# Converte a coluna 'label_alvo' em valores numéricos (por exemplo: aprovado = 0, reprovado = 1)
X['label_alvo'] = le.fit_transform(X['label_alvo'])

# Normaliza os dados (média = 0, desvio padrão = 1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Cria e treina um novo modelo KMeans com 4 clusters
km = KMeans(n_clusters=4, random_state=42)
clusters = km.fit_predict(X_scaled)

# Adiciona o grupo (cluster) encontrado ao novo DataFrame
df_new['grupo'] = clusters

# Calcula a inércia (soma das distâncias dos pontos aos centróides)
inertia = km.inertia_

# Calcula o coeficiente de silhueta (mede a qualidade dos clusters)
silhouette = silhouette_score(X_scaled, clusters)

In [44]:
# Exibe a inércia do modelo (quanto menor, melhor o agrupamento)
print(f'Inércia: {inertia:.2f}')

# Exibe o coeficiente de silhueta (quanto mais próximo de 1, melhor o agrupamento)
print(f'Coeficiente de Silhueta: {silhouette:.4f}')

Inércia: 30678.35
Coeficiente de Silhueta: 0.2204


# Sistema de Recomendação

#### **Objetivo**
1.  Sistemas de recomendação são frequentemente implementados usando técnicas de aprendizado supervisionado, não supervisionado ou baseados em conteúdo.
2. Vamos recomendar a casa para o aluno, baseado na sua idade.



In [45]:
# Gera um novo DataFrame com 1000 registros usando a função 'generate'
df = generate(1000)

# Exibe o DataFrame gerado
df

,aluno,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,label_alvo
0,Kimberly Casey,21,Grifinória,2022,83.80,59.26,71.53,aprovado
1,Fernando Lara,27,Sonserina,2009,66.93,86.22,76.58,aprovado
2,Sarah Long,12,Corvinal,2015,89.86,51.94,70.90,aprovado
3,Stephanie Johnson,35,Grifinória,1999,82.92,1.94,42.43,reprovado
4,James Steele,28,Grifinória,2003,73.49,61.88,67.69,aprovado
...,...,...,...,...,...,...,...,...
995,Wendy Jordan,25,Corvinal,2017,66.48,65.64,66.06,aprovado
996,Cindy Phillips,26,Grifinória,2017,98.29,5.16,51.73,aprovado
997,Melanie Edwards,23,Sonserina,2013,64.77,91.87,78.32,aprovado
998,Latasha Hansen,14,Corvinal,2011,38.23,0.00,38.23,reprovado


In [46]:
# Filtra o DataFrame para manter apenas os alunos aprovados
# A ideia é focar a recomendação apenas em quem teve sucesso (aprovado)
df = df.loc[df['label_alvo'] == 'aprovado']

In [47]:
# Seleciona apenas as variáveis de interesse para o modelo (idade e ano de ingresso)
X = df[['idade', 'ano_de_ingresso']]

# Define a variável alvo como a coluna 'casas'
y = df['casas']

In [48]:
# Transforma a coluna 'casas' de categorias para valores numéricos
# Isso é necessário porque muitos algoritmos de machine learning trabalham apenas com números
le = LabelEncoder()
y = le.fit_transform(y)

In [49]:
# Cria o modelo de vizinhos mais próximos usando o algoritmo 'brute'
# 'brute' força a busca exaustiva (útil para bases pequenas ou médias)
vizinho = NearestNeighbors(algorithm='brute')

# Treina (ajusta) o modelo com os dados de entrada (X) e rótulos (y)
vizinho.fit(X, y)

NearestNeighbors(algorithm='brute')

#### Salvando o modelo

In [50]:
# Cria uma nova pasta chamada 'recomendacao' para salvar o modelo
folder = 'recomendacao'
os.mkdir(folder)

# Salva o modelo treinado de Nearest Neighbors no arquivo 'recomendacao.pkl'
with open(f'{folder}/recomendacao.pkl', 'wb') as f:
  joblib.dump(vizinho, f)

#### Importando o modelo

In [51]:
#importando o modelo
viz = joblib.load(f'{folder}/recomendacao.pkl')

####testando

In [52]:
# Definindo os dados de um novo aluno para recomendação
aluno = 'Daniel Ferreira'
idade = 37
ano_de_ingresso = 2024

In [53]:
# Criar um novo DataFrame com os dados do novo aluno
# Importante: os dados precisam estar em formato de DataFrame para o modelo aceitar
novo_aluno = pd.DataFrame({
  'idade': [idade],
  'ano_de_ingresso': [ano_de_ingresso]
})

In [54]:
# Criar e treinar o modelo Nearest Neighbors
viz.fit(X, y)

NearestNeighbors(algorithm='brute')

In [55]:
# Encontrar os vizinhos mais próximos do novo aluno
# 'kneighbors' retorna as distâncias e os índices dos vizinhos mais próximos
distancias, indices_vizinhos = vizinho.kneighbors(novo_aluno)

In [56]:
# Obter as casas dos vizinhos mais próximos
# Usando os índices dos vizinhos para acessar as casas correspondentes
casas_vizinhos = df.iloc[indices_vizinhos[0]]['casas'].values

In [57]:
# Encontrar a casa mais comum entre os vizinhos
# Transformar a lista de casas dos vizinhos em uma Série do pandas
# Usar o método 'mode()' para encontrar o valor mais frequente (a casa mais popular)
# [0] é usado porque 'mode()' retorna uma Série, e queremos apenas o primeiro valor
casa_recomendada = pd.Series(casas_vizinhos).mode()[0]

In [58]:
# Exibir a recomendação final
# Usamos .upper() para deixar o nome da casa em letras maiúsculas
print(f"Recomendamos para o {aluno} a casa {casa_recomendada.upper()}")

Recomendamos para o Daniel Ferreira a casa CORVINAL
